<a href="https://colab.research.google.com/github/Hena-123/095_HenaPatel/blob/main/Lab7/Lab7_LogisticRegression_Tweets_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hena Patel**

---


**(CE095-18CEUOS114)**

## **Exercise-1**
## **Aim:**
* Extract features for logistic regression given some text
* Implement logistic regression from scratch
* Apply logistic regression on a natural language processing task
* Test logistic regression
* We will be using a data set of tweets.
* Use Tensorflow

In [ ]:
import nltk
from nltk.corpus import twitter_samples 
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [ ]:
#process_tweet(): cleans the text, tokenizes it into separate words, removes stopwords, and converts words to stems.
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
 
    tweet = re.sub(r'#', '', tweet)

    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
            
            #############################################################
            # 1 remove stopwords
            if(word not in stopwords_english and word not in string.punctuation):
              steam_word=stemmer.stem(word)
              tweets_clean.append(steam_word)
            # 3 stemming word
            # 4 Add it to tweets_clean
    return tweets_clean


In [ ]:
def build_freqs(tweets, ys):

    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}

    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            
            #############################################################
            #Update the count of pair if present, set it to 1 otherwise
            if pair not in freqs:
              freqs[pair]=0
            freqs[pair]+=1

    return freqs

Prepare the data

The twitter_samples contains subsets of 5,000 positive tweets, 5,000 negative tweets, and the full set of 10,000 tweets.

In [ ]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

Divide data into train and test

In [ ]:
# combine positive and negative labels
data=all_positive_tweets+all_negative_tweets
la=np.append(np.ones((len(all_positive_tweets), 1)), np.zeros((len(all_negative_tweets), 1)), axis=0)
train_x,test_x,train_y,test_y=train_test_split(data,la,test_size=0.30,random_state=121)

Create the frequency dictionary using build_freqs() function.

In [ ]:
freqs = build_freqs(train_x,train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 10519


Process Tweet

In [ ]:
# Example
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 @moontwink one of the best couples :(

This is an example of the processed version of the tweet: 
 ['one', 'best', 'coupl', ':(']


In [ ]:
def extract_features(tweet, freqs):
 
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 2)) 
    
    #bias term is set to 1
    # x[0,0] = 1 
        
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        #############################################################
        if((word,1) in freqs):
          x[0,0]+=freqs[word,1]
        
        # increment the word count for the negative label 0
        #############################################################
        if((word,0) in freqs):
          x[0,1]+=freqs[word,0]
    
    assert(x.shape == (1, 2))
    return x[0]

In [ ]:
# Check the function

# test 1
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[ 146. 3333.]


In [ ]:
# test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('happy', freqs)
print(tmp2)

[137.  20.]


In [ ]:
def predict_tweet(tweet):

    with tf.Session() as sess:
      saver.restore(sess,save_path='TSession')
      data_i=[]
      for t in tweet:
        data_i.append(extract_features(t,freqs))
      data_i=np.asarray(data_i)
      return sess.run(tf.nn.sigmoid(tf.add(tf.matmul(a=data_i,b=W,transpose_b=True),b)))
    print("Fail")
    return

In [ ]:
b=tf.Variable(np.random.randn(1),name="Bias")
W=tf.Variable(np.random.randn(1,2),name="Bias")

In [ ]:
data=[]
for t in train_x:
  data.append(extract_features(t,freqs))
data=np.asarray(data)

In [ ]:
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(np.asarray(data), W,transpose_b=True), b)) 
print(Y_hat)
ta=np.asarray(train_y)
# Sigmoid Cross Entropy Cost Function 
cost = tf.nn.sigmoid_cross_entropy_with_logits( 
                    logits = Y_hat, labels = ta) 
print(cost)

Tensor("Sigmoid:0", shape=(7000, 1), dtype=float64)
Tensor("logistic_loss:0", shape=(7000, 1), dtype=float64)


In [ ]:
# Gradient Descent Optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-4,name="GradientDescent").minimize(cost) 
  
# Global Variables Initializer 
init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
  
  sess.run(init)
  print("Bias",sess.run(b))
  print("Weight",sess.run(W))
  for epoch in range(400):
    sess.run(optimizer)
    preds=sess.run(Y_hat)
    acc=((preds==ta).sum())/len(train_y)
    accu=[]
    repoch=False
    if repoch:
      accu.append(acc)
    if epoch % 1000 == 0:
      print("Accuracy",acc)
    saved_path = saver.save(sess, 'TSession')

Bias [1.0030144]
Weight [[ 0.02522508 -1.30884557]]
Accuracy 0.9094285714285715


In [ ]:
preds=predict_tweet(test_x)
print(preds,len(test_y))

INFO:tensorflow:Restoring parameters from TSession
[[1.]
 [1.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]] 3000


In [ ]:
def calculate_accuracy(x,y):
  if len(x)!=len(y):
    print("dimensions are different")
    return
  return ((x==y).sum())/len(y)

In [ ]:
print(calculate_accuracy(preds,test_y))

0.9176666666666666
